<a href="https://colab.research.google.com/github/akhi07rx/Youtube-To-Google-Drive/blob/main/Youtube_To_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube

In [ ]:
from tqdm import tqdm
import requests
from google.colab import drive
import os
from urllib.parse import urlparse, unquote
import os.path
import sys
import pytube

drive.mount('/content/drive')

# Define the destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/DOWNLOADS 02/'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Function to download YouTube video
def download_youtube_video(url, destination_folder):
    try:
        youtube = pytube.YouTube(url)
        video = youtube.streams.first()
        video.download(destination_folder)
        print(f"Video '{video.title}' downloaded successfully to '{destination_folder}'.\n")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

try:
    while True:
        try:
            user_input = input("\n\nEnter the download link (or 'exit' to quit): ")
        except KeyboardInterrupt:
            print("\nKeyboard interrupt detected. Exiting the program.")
            break

        if user_input.lower() == 'exit':
            break

        # Check if the input is a YouTube link
        if "youtube.com" in user_input or "youtu.be" in user_input:
            download_youtube_video(user_input, destination_folder)
            continue

        try:
            response = requests.get(user_input, stream=True)
            response.raise_for_status()
        except requests.exceptions.MissingSchema:
            print("You have entered a wrong command. Please enter again.\n")
            continue

        file_size = int(response.headers.get('content-length', 0))

        file_size_mb = file_size / (1024 ** 2)
        file_size_gb = file_size / (1024 ** 3)

        file_size_str = f"{file_size_mb:.2f} MB / {file_size_gb:.2f} GB"

        progress_bar = tqdm(total=file_size, unit='B', unit_scale=True, leave=True)

        parsed_url = urlparse(user_input)
        file_name = unquote(os.path.basename(parsed_url.path))

        file_path = os.path.join(destination_folder, file_name)

        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    progress_bar.update(len(chunk))
                    sys.stdout.flush()

        progress_bar.close()

        print()
        print(f"File name: {file_name}")
        print(f"File size: {file_size_str}")
        print(f"File '{file_name}' uploaded successfully to '{destination_folder}'.\n")

except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Exiting the program.")

print("Exiting the program.")
